# 텐서플로우 즉시 실행 (TensorFlow Eager Execution)
텐서플로우의 즉시 실행 (Eager execution)은 그래프 생성 없이 연산을 즉시 실행하는 명령형 프로그래밍 환경을 뜻한다. 각 연산들은 나중에 실행할 계산 그래프를 만드는 것이 아니라, 실제 값이 반환된다. 이를 통해 텐서플로우를 좀더 쉽게 시작할 수 있고, 모델을 디버그 할 수 있다. 또한 불필요한 구문도 줄여준다. 

* 직관적인 인터페이스—사용자 코드를 자연스럽게 구조화 하고, 파이썬 데이터 구조를 사용한다. 작은 모델과 작은 데이터에 대해서도 빠르게 반복수행 가능하다.
* 쉬운 디버깅—실행중인 모델을 검사하거나 변화사항을 평가할 때 연산들을 직접 호출할 수 있다.
* 자연스러운 흐름 제어—동적 모델의 명세를 단순화 시켜, 그래프 흐름 제어 대신 파이썬 흐름 제어를 사용할 수 있다.  

즉시 실행 (eager execution)은 텐서플로우의 대부분 연산 및 GPU 가속화를 지원한다.  

즉시 실행 (eager execution)을 시작하기 위해서, tf.enable_eager_execution() 구문을 프로그램이나 콘솔세션 제일 첫 부분에 추가한다.

In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


###  Without Eager Execution
```
tf.enable_eager_execution()

tf.executing_eagerly()        # => True

a = tf.constant([[1.,2.], [3.,4.]])
print(a)
print(tf.matmul(a,a))
```
수행 결과는 다음과 같다.

```
Tensor("Const:0", shape=(2, 2), dtype=float32)
Tensor("MatMul:0", shape=(2, 2), dtype=float32)
```
Eager Execution을 쓰지 않을 경우에는 다음과 같이 session을 수행해야 원하는 결과를 확인할 수 있다.

```
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(tf.matmul(a,a)))
```
```
[[1. 2.]
 [3. 4.]]
[[ 7. 10.]
 [15. 22.]]
```


### With Eager Execution 
즉시 실행 (eager execution)을 활성화하여 텐서플로우 연산이 즉시 실행되어 파이썬에게 그 값을 반환하여 줄 수 있도록 동작을 바꾸게 된다.

In [2]:
tf.enable_eager_execution()

tf.executing_eagerly()        # => True

True

In [3]:
a = tf.constant([[1.,2.], [3.,4.]])
print(a)
print(tf.matmul(a,a))

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 7. 10.]
 [15. 22.]], shape=(2, 2), dtype=float32)


# Numpy 호환성
즉시 실행 (eager execution)은 NumPy와 호환성이 매우 뛰어나다. 
* NumPy 연산은 tf.Tensor를 인자로 받는다. 
* 텐서플로우 수학 연산은 파이썬 객체와 NumPy 배열을 tf.Tensor 객체로 변환한다. 
* tf.Tensor.numpy 함수는 객체의 값을 NumPy ndarray형태로 반환합니다.

In [4]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)
# => tf.Tensor([[1 2]
#               [3 4]], shape=(2, 2), dtype=int32)

# 브로드캐스팅을 지원합니다.
b = tf.add(a, 1)
print(b)
# => tf.Tensor([[2 3]
#               [4 5]], shape=(2, 2), dtype=int32)

# 연산자 오버로딩을 지원합니다.
print(a * b)
# => tf.Tensor([[ 2  6]
#               [12 20]], shape=(2, 2), dtype=int32)

# NumPy 값을 써봅시다.
import numpy as np

c = np.multiply(a, b)
print(c)
# => [[ 2  6]
#     [12 20]]

# 텐서로부터 numpy 형태의 값 받기:
print(a.numpy())
# => [[1 2]
#     [3 4]]

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)
[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


# 동적 흐름 제어 (Dynamic control flow)

### Without Eager Execution
```
b = tf.constant([1.,2.,3.])
for i in b:
    print(i)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-16-f72bc6bf22c1> in <module>
      2 
      3 b = tf.constant([1.,2.,3.])
----> 4 for i in b:
      5     print(i)
```

### With Eager Execution
파이썬처럼 자연스러운 동적 흐름 제어가 가능하다.

In [5]:
b = tf.constant([1.,2.,3.])
for i in b:
    print(i)

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)


# 즉시 학습 (Eager training)

### 그래디언트 계산하기 - 자동미분
즉시 실행 (eager execution)이 수행되는 동안, tf.GradientTape를 이용하여 나중에 gradient 계산을 수행할 연산을 추적할 수 있다.

정방향(forward-pass) 연산은 "tape"에 기록됩니다. 그다음 tape를 거꾸로 돌려 그래디언트를 계산한 후 tape를 폐기한다.

In [6]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, w)
print(grad)  # => tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


tf.Variable 객체는 자동 미분을 쉽게 하기 위해서 학습동안 변경된 tf.Tensor 값을 저장한다.  

tf.Variable을 tf.GradientTape과 함께 사용하여 모델을 생성한다.

In [7]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')
    def call(self, inputs):
        return inputs * self.W + self.B

# 약 3 * x + 2개의 점으로 구성된 실험 데이터
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# 최적화할 손실함수
def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

# 정의:
# 1. 모델
# 2. 모델 파라미터에 대한 손실 함수의 미분
# 3. 미분에 기초한 변수 업데이트 전략
model = Model()
#optimizer = tf.keras.optimizers.SGD(lr=0.01)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("초기 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# 반복 훈련
for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]))
    if i % 50 == 0:
        print("스텝 {:03d}에서 손실: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("최종 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

초기 손실: 68.937
스텝 000에서 손실: 66.256
스텝 050에서 손실: 9.704
스텝 100에서 손실: 2.147
스텝 150에서 손실: 1.137
스텝 200에서 손실: 1.002
스텝 250에서 손실: 0.984
최종 손실: 0.981
W = 2.9780404567718506, B = 2.0127627849578857
